In [3]:
api_key = 'RGAPI-cb129971-94a0-43ea-93c5-60c4fad7ae5a'

In [4]:
import requests

In [7]:
r = requests.get("https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/clu/clu1?api_key=" + api_key)
print(r.json())

{'puuid': 'R8_dzuvxZm0VsqU-paXM3wijLavgPmiZDmKIHFm8HddnUGvsgHTlCQuWdlHNRJlBIaEFyZYD87Avug', 'gameName': 'clu', 'tagLine': 'clu1'}


In [5]:
# setsuko, soju, dishsoap
streamers = ['e62ZthNQBc1bjVfXrjDvNIQV-0Bd7GUeaRv_fiZqx5nc5OXaY6OAmM4wav043AJgxKQShtA1s9IIOg', 'hVi-4fc19ymaRf4HvUmHNkZCoALJPUXgGKcKGgBlLZTqYZRVtQwglGQCIBj0jou0wA35CcH8sdeEgw', 'NUnja2Bl0CPRiimqr1PesnOnDsxXa6eAg5xRhNVBdxgAzBopnpcWNTZZ9r9YjNhEtf5tx2DkvXdJ9Q']

In [9]:
# for streamer_puuid in streamers:
#   matches = requests.get("https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/" + streamer_puuid + "&api_key=" + api_key)


In [2]:
# connect to db
import os
import asyncio
import asyncpg
from dotenv import load_dotenv

async def main():
    # Load .env file
    load_dotenv()

    # Get the connection string from the environment variable
    connection_string = os.getenv('DATABASE_URL')

    # Create a connection pool
    pool = await asyncpg.create_pool(connection_string)

    # Acquire a connection from the pool
    async with pool.acquire() as conn:
        # Execute SQL commands to retrieve the current time and version from PostgreSQL
        time = await conn.fetchval('SELECT NOW();')
        version = await conn.fetchval('SELECT version();')

    # Close the pool
    await pool.close()

    # Print the results
    print('Current time:', time)
    print('PostgreSQL version:', version)

# Run the asynchronous main function
await(main())

Current time: 2024-10-01 20:08:22.411854+00:00
PostgreSQL version: PostgreSQL 16.4 on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit


In [17]:
puuid = streamers[0]

# get recent match data
matches = requests.get('https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/' + puuid + '/ids?start=0&count=20&api_key=' + api_key).json()
# print(matches)

# check that matches are not already in db
# later :D:D:D

# get the stuff
# define carries and tier
carries = []

for match in matches:
  match_data = requests.get('https://americas.api.riotgames.com/tft/match/v1/matches/' + match + '?api_key=' + api_key).json()

  participants = match_data['metadata']['participants']
  patch = match_data['info']['game_version'][-6:-1]
  game_datetime = match_data['info']['game_datetime']

  game_information = match_data['info']['participants']
  for board in game_information:
    augments = board['augments']
    level = board['level']
    placement = board['placement']
    traits = board['traits']
    units = board['units']
    # figure out what comp they are playing

    
  # print(match_data)




  break


['TFT11_Augment_Epoch', 'TFT12_Augment_Faerie', 'TFT_Augment_RanduinsImmovableObject']
['TFT_Augment_DoubleDown', 'TFT9_Augment_OldMansWalkingStick', 'TFT12_Augment_ScholarCrown']
['TFT_Augment_DoubleDown', 'TFT12_Augment_Arcana', 'TFT9_Augment_Commander_FinalAscension']
['TFT11_Augment_Slammin', 'TFT_Augment_Replication', 'TFT11_Augment_Calltochaos']
['TFT12_Augment_BlasterCrest', 'TFT9_Augment_BalancedBudget2', 'TFT7_Augment_LuckyGlovesPlus']
['TFT10_Augment_CrownGuarded', 'TFT12_Augment_HunterCrest', 'TFT_Augment_GiantAndMighty']
['TFT12_Augment_Frost', 'TFT9_Augment_YouHaveMyBow', 'TFT12_Augment_FrostCrown']
['TFT6_Augment_TradeSector', 'TFT10_Augment_LittleBuddies', 'TFT_Augment_GiantAndMighty']
